# Lab 4 report

## Setting up velocity controllers

ผู้จัดทำได้ตั้งค่า velocity controller โดยใช้ velocity_controllers/JointGroupVelocityController โดยมี command interface ซึ่งเป็น input ของระบบเป็น velocity และตั้งค่า state interface ซึ่งเป็น feedback ของแต่ละ joint ได้แก่ position, velocity

<img src="pic/ros2control_param.png" width="500">

จากนั้นจึงตั้งค่า ros2control ในไฟล์ urdf

<img src="pic/ros2control_gazebo.png">

## Developing a tracker

สร้าง tracker.py โดย subscribe topic ได้แก่ /dynamic_joint_states เพื่อนำ feedback ของแต่ละ joint มาใช้ในหาร control

<img src="pic/dynamic_joints_state.png">

โดยใน tracker.py ผู้จัดทำได้สร้าง method ในการดึงค่า position feedback จาก dynamic_joint_states และสร้าง jacobian matrix

<img src="pic/joint_feedback.png">

โดยระบบ control จะใช้ parameter ที่อ่านได้ yaml

<img src="pic/controller_param.png">

โดย tracker.py จะดึงค่า parameter ตอนสร้าง node

## Modifying kinematics server

สร้าง controller loop โดยทำ forward control ด้วยความเร็วเชิงเส้นของปลายแขนและทำ feedback control ด้วย joint config feedback

<img src="pic/control_loop.png">

เมื่อได้ control loop แล้ว ผู้จัดทำได้สร้าง FSM เพื่อกำหนดสถานะของหุ่นยนต์โดยมีสถานะได้แก่
1. SET_HOME ทำหน้าที่พาหุ่นยนต์ไปเริ่มต้นที่จุดที่ไม่เกิด singularity
2. FORWARD ทำหน้าที่พาหุ่นยนต์ให้เดินตาม trajectory เมื่อถึงจุหมายจะ service call ไปที่ scheduler.py เพื่อให้ส่งตำแหน่งปลายทางกลับมา
3. WAIT ทำหน้าที่คงตำแหน่งของหุ่นยนต์ระหว่าที่รอ trajectory ใหม่
โดย method ตัวนี้คือ timer_callback ซึ่งมีความถี่ในการควบคุม 100 Hz

## Developing a generator

ในส่วน gererator.py ได้ใช้ ActionServer ในการคำนวณหา trajectory ที่เร็วที่สุดโดยกำหนด velocity contrain และ acceleration constrain เนื่อวจากใช้ while loop ในการหาเวลาที่เร็วที่สุดเราจึงใช้ callback ของ ActionServer ในการรอให้ while loop ทำงานเสร็จจะได้ไม่เกิดการ blocking เมื่อคำยวณเสร็จ ActionServer จะส่งค่าสัมประสิทและเวลาที่ใช่ในการเคลื่อนที่กลับไปที่ tracker.py

<img src="pic/action_structure.png">

method สำหรับคำนวณ trajectory จะทำงานเมื่อ tracker.py ได้รับ service call จาก scheduler.py ตัวของ tracker.py จะ send_goal มาที่ generator.py

<img src="pic/trajectory_compute.png">

## Developing a proximity detector

การที่จะให้หุ่นยนต์สามารถรับรู้ว่าถึงตำแหน่งแล้วเราจะใช้ event driven ก็คือการที่เราตรวจว่าปลายของ end effector ห่างจากจุดหายปลายทางตำกว่าค่า threshold แล้วจะถือว่าถึง โดยเราตั้งว่าถ้าปลายของ end effector ห่างจากจุดหายปลายทาง 1 มิลลิเมตร จะถือว่าถึงแล้ว โดยใช้ euclidian distant ในการวัดระยะ

<img src="pic/proximity.png">

## Developing a scheduler

scheduler.py จะเก็บเส้นที่ใช้เขียนตัวอักษร โดยจะเช็กว่าถ้าหางเส้นที่แล้วคือหัวของเส้นถัดไปจะเขียนต่อ แต่ถ้าไม่จะทำการยกปากกา โดยจะส่งพิกัดใหม่ไปที่ tracker.py เมื่อมี service call มาจาก tracker.py โดยเก็บเส้นในรูปแบบหัวและหางของเส้น

<img src="pic/line.png">

method ที่ใช้ในการเลือกว่าจะวาดต่อเส้นหรือยกปากกา

<img src="pic/optimize_line.png">